In [1]:
import time
start_time = time.time()
from qiskit_ibm_provider import IBMProvider
provider = IBMProvider()
# Standard imports
import numpy as np
import pandas as pd
import time
import warnings

# Qiskit 
from qiskit import QuantumCircuit, transpile
from qiskit.circuit import Parameter
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes, EfficientSU2
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier, VQC
from qiskit_machine_learning.algorithms.regressors import NeuralNetworkRegressor, VQR
from qiskit_machine_learning.neural_networks import SamplerQNN, EstimatorQNN
from qiskit_machine_learning.circuit.library import QNNCircuit
from qiskit.primitives import Sampler
from qiskit_algorithms.optimizers import COBYLA, L_BFGS_B, ADAM, SLSQP, AQGD
from qiskit_algorithms.utils import algorithm_globals

# Visualization libraries
from matplotlib import pyplot as plt
import seaborn as sns
from qiskit.visualization import plot_histogram
from IPython.display import clear_output

# Machine learning and preprocessing
from sklearn.datasets import load_iris, load_digits,load_wine, load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.exceptions import ConvergenceWarning
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA

In [2]:
datasets = [load_iris, load_digits, load_wine, load_breast_cancer]  

def data_preprocess(dataset):
    data = dataset()
    dataset_features = data.data
    dataset_labels = data.target
    dataset_features = MinMaxScaler().fit_transform(dataset_features)
    algorithm_globals.random_seed = 4701
    data_set_num_features = dataset_features.shape[1]
    dataset_train_features, dataset_test_features, dataset_train_labels, dataset_test_labels = train_test_split(
        dataset_features, dataset_labels, train_size=0.8, random_state=algorithm_globals.random_seed)
    
    print(f'{dataset.__name__} preprocessed')
    return dataset_train_features, dataset_test_features, dataset_train_labels, dataset_test_labels, data_set_num_features

In [3]:
for dataset in datasets:
    print(f'{dataset.__name__}')
    train_features, test_features, train_labels, test_labels, num_features = data_preprocess(dataset)
    
    svc = SVC()
    svc.fit(train_features, train_labels)
    
    predictions = svc.predict(test_features)
    accuracy = svc.score(test_features, test_labels)
    
    print(f'{dataset.__name__} SVC Score: {accuracy:.4f}')


load_iris
load_iris preprocessed
load_iris SVC Score: 0.9000
load_digits
load_digits preprocessed
load_digits SVC Score: 0.9917
load_wine
load_wine preprocessed
load_wine SVC Score: 1.0000
load_breast_cancer
load_breast_cancer preprocessed
load_breast_cancer SVC Score: 0.9825


In [4]:
for dataset in datasets:
    train_features, test_features, train_labels, test_labels, num_features = data_preprocess(dataset)
feature_map = ZZFeatureMap(feature_dimension=num_features, reps = 1)
ansatz = RealAmplitudes(num_qubits=num_features, reps = 3)
optimizer = COBYLA(maxiter=100)
sampler = Sampler()

load_iris preprocessed
load_digits preprocessed
load_wine preprocessed
load_breast_cancer preprocessed


In [ ]:
for dataset in datasets:
    train_features, test_features, train_labels, test_labels, num_features = data_preprocess(dataset)
feature_map = ZZFeatureMap(feature_dimension=num_features, reps = 1)
ansatz = RealAmplitudes(num_qubits=num_features, reps = 3)
optimizer = COBYLA(maxiter=100)
sampler = Sampler()
objective_func_vals = []

# objective function characterizes the distance between the predictions and known labeled data.

plt.rcParams['figure.figsize'] = (12,6)

def callback_graph(weights, obj_func_eval):
             
# Append the value of the objective function to an array so we 
# can plot the iteration verses the objective function value
             
             clear_output(wait=True)
             objective_func_vals.append(obj_func_eval)
             plt.title('Objective function value against Iteration')
             plt.xlabel('Iteration')
             plt.ylabel('Objective function value')
             plt.plot(range(len(objective_func_vals)), objective_func_vals)
             
             plt.show()

vqc = VQC(
    sampler = sampler, 
    feature_map = feature_map,
    ansatz = ansatz, 
    optimizer = optimizer, 
    callback = callback_graph,)

# Clear objective value history
objective_func_vals = []

start = time.time()
vqc.fit(train_features,train_labels)
elsapsed_time = time.time() - start 

print(f'Training time: {elsapsed_time}')

load_iris preprocessed
load_digits preprocessed
load_wine preprocessed
load_breast_cancer preprocessed


In [ ]:
vqc = VQC(
    sampler = sampler, 
    feature_map = feature_map,
    ansatz = ansatz, 
    optimizer = optimizer, 
    callback = callback_graph,)

# Clear objective value history
objective_func_vals = []

start = time.time()
vqc.fit(train_features,train_labels)
elsapsed_time = time.time() - start 

print(f'Training time: {elsapsed_time}')

In [ ]:

datasets = [load_iris, load_digits, load_wine, load_breast_cancer]  # Assuming data_preprocess is defined correctly

for dataset_loader in datasets:
    train_features, test_features, train_labels, test_labels, num_features = data_preprocess(dataset_loader())
    feature_map = ZZFeatureMap(feature_dimension=num_features, reps=1)
    ansatz = RealAmplitudes(num_qubits=num_features, reps=3)
    optimizer = COBYLA(maxiter=100)
    sampler = Sampler()
    objective_func_vals = []  # To track objective function values

    def callback_graph(weights, obj_func_eval):
        # Append and plot the objective function value
        clear_output(wait=True)
        objective_func_vals.append(obj_func_eval)
        plt.title('Objective Function Value Against Iteration')
        plt.xlabel('Iteration')
        plt.ylabel('Objective Function Value')
        plt.plot(objective_func_vals)
        plt.show()

    vqc = VQC(sampler=sampler, feature_map=feature_map, ansatz=ansatz, optimizer=optimizer, callback=callback_graph)
    
    # Train the VQC
    start_time = time.time()
    vqc.fit(train_features, train_labels)
    elapsed_time = time.time() - start_time

    print(f'Training time for {dataset_loader.__name__}: {elapsed_time:.2f} seconds\n')

# Note: Ensure that data_preprocess returns the correct values in the expected order
